In [1]:
from datasets import load_dataset

/Users/mohammed.jabir/Desktop/code/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = load_dataset('bigbio/drugprot','drugprot_bigbio_kb')
data

Found cached dataset drugprot (/Users/mohammed.jabir/.cache/huggingface/datasets/bigbio___drugprot/drugprot_bigbio_kb/1.0.0/b8862a4fb4801918002f3e66268f4a58377751de46c9b2b1fc43df10d242d3e7)
100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 192.59it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 3500
    })
    validation: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 750
    })
})

In [3]:
data['train'].features["entities"]

[{'id': Value(dtype='string', id=None),
  'type': Value(dtype='string', id=None),
  'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
  'offsets': Sequence(feature=[Value(dtype='int32', id=None)], length=-1, id=None),
  'normalized': [{'db_name': Value(dtype='string', id=None),
    'db_id': Value(dtype='string', id=None)}]}]

## Single data

### Using NLTK

In [4]:
txt=''
for x in data['train'][0]['passages']:
    for j in x['text']:
        txt+=j+' '
txt

"RDH12, a retinol dehydrogenase causing Leber's congenital amaurosis, is also involved in steroid metabolism. Three retinol dehydrogenases (RDHs) were tested for steroid converting abilities: human and murine RDH 12 and human RDH13. RDH12 is involved in retinal degeneration in Leber's congenital amaurosis (LCA). We show that murine Rdh12 and human RDH13 do not reveal activity towards the checked steroids, but that human type 12 RDH reduces dihydrotestosterone to androstanediol, and is thus also involved in steroid metabolism. Furthermore, we analyzed both expression and subcellular localization of these enzymes. "

In [72]:
from nltk.tokenize import TreebankWordTokenizer as twt
spans_txt = twt().span_tokenize(txt)

In [59]:
entities = [None]*len(list(spans_txt))

In [75]:
start={}
end ={}

for idx ,sp in enumerate(spans_txt):
    start[sp[0]]=idx
    end[sp[1]]=idx

In [77]:
ents =[]
for key , val in data['train'][0].items():
    if key == 'entities':
        for item in data['train'][0]['entities']:
            ent=[]
            ent.append(item['type'])
#             for t in item['text']:
#                 ent.append(t)
            for of in item['offsets']:
                for f in of :
                    ent.append(f)
            ents.append(ent)

In [78]:
ents

[['CHEMICAL', 466, 480],
 ['CHEMICAL', 115, 122],
 ['CHEMICAL', 9, 16],
 ['GENE-Y', 219, 230],
 ['GENE-Y', 232, 237],
 ['GENE-Y', 326, 338],
 ['GENE-Y', 343, 354],
 ['GENE-N', 139, 143],
 ['GENE-Y', 417, 434],
 ['GENE-N', 115, 137],
 ['GENE-N', 191, 214],
 ['GENE-Y', 0, 5],
 ['GENE-N', 9, 30]]

In [79]:
for label , start_char , end_char in ents:
    start_token = start.get(start_char)
    end_token = end.get(end_char)
    if start_token is not None and end_token is not None:
        if start_token == end_token:
            entities[start_token] = f"B-{label}"
        else:
            entities[start_token] = f"B-{label}"
            for i in range(start_token + 1, end_token):
                entities[i] = f"I-{label}"
            entities[end_token] = f"I-{label}"

In [80]:
for idx , e in enumerate(entities):
    if e ==None:
        entities[idx]='o'

In [81]:
for sp , ent in zip(spans_txt,entities):
    print(txt[sp[0]:sp[1]] , ent)

In [82]:
entities

['o',
 'o',
 'B-GENE-N',
 'I-GENE-N',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'B-GENE-N',
 'I-GENE-N',
 'o',
 'B-GENE-N',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'B-GENE-N',
 'I-GENE-N',
 'I-GENE-N',
 'I-GENE-N',
 'I-GENE-N',
 'o',
 'o',
 'o',
 'B-GENE-Y',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'B-GENE-Y',
 'I-GENE-Y',
 'o',
 'B-GENE-Y',
 'I-GENE-Y',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'B-GENE-Y',
 'I-GENE-Y',
 'I-GENE-Y',
 'I-GENE-Y',
 'o',
 'o',
 'o',
 'B-CHEMICAL',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o']

### Using Spacy

In [84]:
import spacy
from spacy.training.iob_utils import biluo_to_iob, doc_to_biluo_tags

In [85]:
nlp = spacy.blank("en")
doc = nlp(txt)

In [86]:
ents = []
for  val in data['train'][0]['entities']:
    start = val['offsets'][0][0]
    end = val['offsets'][0][1]
    span = doc.char_span(start, end, label=val['type'])
    if span is not None:
        ents.append(span)
    else:
        print(
            "Skipping span (does not align to tokens):",
            start,
            end,
            val['type'],
            doc.text[start:end],
        )

In [87]:
ents

[androstanediol,
 retinol,
 retinol,
 human RDH13,
 RDH12,
 murine Rdh12,
 human RDH13,
 RDHs,
 human type 12 RDH,
 retinol dehydrogenases,
 human and murine RDH 12,
 RDH12,
 retinol dehydrogenase]

In [88]:
filtered = spacy.util.filter_spans(ents)

In [89]:
filtered

[RDH12,
 retinol dehydrogenase,
 retinol dehydrogenases,
 RDHs,
 human and murine RDH 12,
 human RDH13,
 RDH12,
 murine Rdh12,
 human RDH13,
 human type 12 RDH,
 androstanediol]

In [90]:
doc.ents = filtered

In [91]:
if doc.ents: 
    for ent in doc.ents: 
        print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
else: print('No named entities found.')

RDH12 - 0 - 5 - GENE-Y - None
retinol dehydrogenase - 9 - 30 - GENE-N - None
retinol dehydrogenases - 115 - 137 - GENE-N - None
RDHs - 139 - 143 - GENE-N - None
human and murine RDH 12 - 191 - 214 - GENE-N - None
human RDH13 - 219 - 230 - GENE-Y - None
RDH12 - 232 - 237 - GENE-Y - None
murine Rdh12 - 326 - 338 - GENE-Y - None
human RDH13 - 343 - 354 - GENE-Y - None
human type 12 RDH - 417 - 434 - GENE-Y - None
androstanediol - 466 - 480 - CHEMICAL - None


/Users/mohammed.jabir/Desktop/code/env/lib/python3.9/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'GENE-Y' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
/Users/mohammed.jabir/Desktop/code/env/lib/python3.9/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'GENE-N' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
/Users/mohammed.jabir/Desktop/code/env/lib/python3.9/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'CHEMICAL' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Wa

In [92]:
for token, iob_tag in zip(doc, biluo_to_iob(doc_to_biluo_tags(doc))):
    print(token.text + " " + iob_tag)

RDH12 B-GENE-Y
, O
a O
retinol B-GENE-N
dehydrogenase I-GENE-N
causing O
Leber O
's O
congenital O
amaurosis O
, O
is O
also O
involved O
in O
steroid O
metabolism O
. O
Three O
retinol B-GENE-N
dehydrogenases I-GENE-N
( O
RDHs B-GENE-N
) O
were O
tested O
for O
steroid O
converting O
abilities O
: O
human B-GENE-N
and I-GENE-N
murine I-GENE-N
RDH I-GENE-N
12 I-GENE-N
and O
human B-GENE-Y
RDH13 I-GENE-Y
. O
RDH12 B-GENE-Y
is O
involved O
in O
retinal O
degeneration O
in O
Leber O
's O
congenital O
amaurosis O
( O
LCA O
) O
. O
We O
show O
that O
murine B-GENE-Y
Rdh12 I-GENE-Y
and O
human B-GENE-Y
RDH13 I-GENE-Y
do O
not O
reveal O
activity O
towards O
the O
checked O
steroids O
, O
but O
that O
human B-GENE-Y
type I-GENE-Y
12 I-GENE-Y
RDH I-GENE-Y
reduces O
dihydrotestosterone O
to O
androstanediol B-CHEMICAL
, O
and O
is O
thus O
also O
involved O
in O
steroid O
metabolism O
. O
Furthermore O
, O
we O
analyzed O
both O
expression O
and O
subcellular O
localization O
of O
these O
enzym